# Python Walkthrough 1.1

## Importing the datafile into Python

We want to import the datafile called ‘NH.Ts+dSST.csv’ from NASA's website into Python using Visual Studio Code.

We start by opening Visual Studio Code in the folder we'll be working in. Use File -> Open Folder to do this. We also need to ensure that the interactive Python window that we'll be using to run Python code opens in this folder. In Visual Studio Code, you can ensure that the interactive window starts in the folder that you have open by setting “Jupyter: Notebook File Root” to `${workspaceFolder}` in the Settings menu.

Now create a new file (File -> New File in the menu) and name it `exercise_1.py`. In the new and empty file, right-click and select "Run file in interactive window". This will launch the interactive Python window.

You will need to install the following packages: **pandas**, **numpy** and **matplotlib**. Packages are add-ons that extend the functionality of the Python language. To install packages, hit use the <kbd>⌘</kbd> + <kbd>\`</kbd> keyboard shortcut (Mac) or <kbd>ctrl</kbd> + <kbd>\`</kbd> (Windows/Linux), or click "View > Terminal". A new box, called the terminal, should appear at the bottom of the screen. To install packages, type `pip install packagename` into this box.

**pandas**, **numpy** and **matplotlib** provide extra functionality for data analysis, numbers, and plotting respectively.

We will download the data directly from the internet into our Python session using the **pandas** library (which provides data manipulation in Python).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.read_csv(
    "https://data.giss.nasa.gov/gistemp/tabledata_v4/NH.Ts+dSST.csv",
    skiprows=1,
    na_values="***",
)

When using the `read_csv` function, we added two options. If you open the spreadsheet in Excel, you will see that the real data only starts in Row 2, so we use the `skiprows = 1` option to skip the first row when importing the data. When looking at the spreadsheet, you can see that missing temperature data is coded as `***`. In order to ensure that the missing temperature data are recorded as numbers, we tell **pandas** that `na_values = "***"` which imports those missing values as `NaN`, which means the number is missing.

To check that the data have been imported correctly, you can use the `.head()` function to view the first five rows of the dataset, and confirm that they correspond to the columns in the csv file.

In [ ]:
df.head()

Before working with this data, we use the `.info()` function to check that the data were read in as numbers (either real numbers, `float64`, or integers, `int`) rather than strings.

In [ ]:
df.info()

You can see that all variables are formatted as either `float64` or `int`, so Python correctly recognises that these data are numbers.

## Python Walkthrough 1.2

**Drawing a line chart of temperature and time**

We will now set the year as the *index* of the dataset. This will make plotting the time series of temperature easier.

In [ ]:
df = df.set_index("Year")
df.head()

In [ ]:
df.tail()

Next we'll make our chart. We'll use the **matplotlib** package for this. The built-in style isn't very attractive so we'll first switch to a prettier one:

In [ ]:
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams["figure.dpi"] = 150

We can now use these variables to draw line charts using the plot function. As an example, we will draw a line chart using data for January, `df["Jan"]` for the years 1880—2016. 

The first line creates a variable for the month (in case we'd like to replot a different month later), the second creates an empty chart, the next two create a horizontal line and annotate it. Then `df[month].plot(ax=ax)` adds the temperature data to the chart. Finally, the title and y-axis label are set. We ensure that the month and year used in the title is always up to date and consistent with the data that we're pulling in by using the variables `month` and `df.index.max()` to write the month and maximum year into the title respectively. (This trick is called an f-string, or function string, and the string begins with an `f` before the opening of the quotation marks.)

In [ ]:
month = "Jan"
fig, ax = plt.subplots()
ax.axhline(0, color="orange")
ax.annotate("1951—1980 average", xy=(0.66, -0.2), xycoords=("figure fraction", "data"))
df[month].plot(ax=ax)
ax.set_title(
    f"Average temperature anomaly in {month} \n in the northern hemisphere (1880—{df.index.max()})"
)
ax.set_ylabel("Annual temperature anomalies");

Try different values for `color`, `xy`, and the first argument of `ax.axhline` in the plot function to figure out what these options do. (Note that `xycoords` set the behaviour of `xy`.)

It is important to remember that all axis and chart titles should be enclosed in quotation marks (`""`), as well as any words that are not options (for example, colour names or filenames).

## Python Walkthrough 1.3

**Producing a line chart for the annual temperature anomalies**

This is where the power of programming languages becomes evident: to produce the same line chart for a different variable, we simply take the code used in Python walk-through 1.2 and replace the `"Jan"` with the name for the annual variable (`"J-D"`).

In [ ]:
month = "J-D"
fig, ax = plt.subplots()
ax.axhline(0, color="orange")
ax.annotate("1951—1980 average", xy=(0.68, -0.2), xycoords=("figure fraction", "data"))
df[month].plot(ax=ax)
ax.set_title(
    f"Average annual temperature anomaly in \n in the northern hemisphere (1880—{df.index.max()})"
)
ax.set_ylabel("Annual temperature anomalies");

## Python Walkthrough 1.4

**Creating frequency tables and histograms**

Since we will be looking at data from different subperiods (year intervals) separately, we will create a categorical variable (a variable that has two or more categories) that indicates the subperiod for each observation (row). In Python this type of variable is called a ‘category’ or categorical. When we create a categorical column, we need to define the categories that this variable can take.

We'll achieve this using a dictionary, a special type of object in Python that provides a mapping from one value to another

In [ ]:
df["Period"] = pd.cut(
    df.index,
    bins=[1921, 1950, 1980, 2010],
    labels=["1921—1950", "1951—1980", "1981—2010"],
    ordered=True,
)

We created a new variable called `"Period"` and defined the possible categories using the `labels=` keyword argument. Since we will not be using data for some years (before 1921 and after 2010), we want `"Period"` to take the value `NaN` (not a number) for these observations (rows), and the appropriate category for all the other observations. The `pd.cut` function does this automatically.

Let's take a look at the last 20 entries using `.tail`:

In [ ]:
df["Period"].tail(20)

We'd really like to combine the data from the three summer months. This is easy to do using the `.stack` function. Let's look at the first few rows of the data once stacked using `.head()`

In [ ]:
list_of_months = ["Jun", "Jul", "Aug"]
df[list_of_months].stack().head()

We need to use all monthly anomalies from June, July, and August, but they are currently in three separate columns.

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(9, 4), sharex=True, sharey=True)
for ax, period in zip(axes, df["Period"].dropna().unique()):
    df.loc[df["Period"] == period, list_of_months].stack().hist(ax=ax)
    ax.set_title(period)
plt.suptitle("Histogram of temperature anomalies")
axes[1].set_xlabel("Summer temperature distribution")
plt.tight_layout();

To explain what a histogram displays, let's take a look at the histogram for the period from 1921—1950. We will look first at the highest of the bars, which is centred at –0.15. This bar represents values of the temperature anomalies that fall in the interval from –0.2 to –0.1. The height of this bar is a representation of how many values fall into this interval, (23 observations, in this case). As it is the highest bar, this indicates that this is the interval in which the largest proportion of temperature anomalies fell for the period from 1921 to 1950. As you can see, there are virtually no temperature anomalies larger than 0.3. The height of these bars gives a useful overview of the distribution of the temperature anomalies.

Now consider how this distribution changes as we move through the three distinct time periods. The distribution is clearly moving to the right for the period 1981–2010, which is an indication that the temperature is increasing; in other words, an indication of global warming.


## Python Walkthrough 1.5

**Using the `quantile` function**

First, we need to create a variable that contains all monthly anomalies in the years 1951—1980. Then, we'll use **pandas** `pd.qcut` function to find the required percentiles (0.3 and 0.7 refer to the 3rd and 7th deciles, respectively).

Note: You may get slightly different values to those shown here if you are using the latest data.

In [ ]:
# Create a variable that has years 1951 to 1980, and months Jan to Dec (inclusive)
temp_all_months = df.loc[(df.index >= 1951) & (df.index <= 1980), "Jan":"Dec"]
# Put all the data in stacked format and give the new columns sensible names
temp_all_months = (
    temp_all_months.stack()
    .reset_index()
    .rename(columns={"level_1": "month", 0: "values"})
)
# Take a look at this data:
temp_all_months

In [ ]:
quantiles = [0.3, 0.7]
list_of_percentiles = np.quantile(temp_all_months["values"], q=quantiles)

print(f"The cold threshold of {quantiles[0]*100}% is {list_of_percentiles[0]}")
print(f"The hot threshold of {quantiles[1]*100}% is {list_of_percentiles[1]}")

# Python Walkthrough 1.6

**Computing the proportion of anomalies at a given quantile using the `.mean()` function**

Note: You may get slightly different values to those shown here if you are using the latest data.

We repeat the steps used in Python walk-through 1.5, now looking at monthly anomalies in the years 1981—2010. We can simply change the year values in the code from Walkthrough 1.5.


In [ ]:
# Create a variable that has years 1981 to 2010, and months Jan to Dec (inclusive)
temp_all_months = df.loc[(df.index >= 1981) & (df.index <= 2010), "Jan":"Dec"]
# Put all the data in stacked format and give the new columns sensible names
temp_all_months = (
    temp_all_months.stack()
    .reset_index()
    .rename(columns={"level_1": "month", 0: "values"})
)
# Take a look at the start of this data data:
temp_all_months.head()

Now that we have all the monthly data for 1981—2010, we want to count the proportion of observations that are smaller than –0.1. We'll first create a *binary indicator* (ie it's True or False) that says, for each row (observation) in `temp_all_months`, whether the number is lower than the 0.3 quantile or not (given by `list_of_percentiles[0]`). Then we'll take the mean of this list of True and False values; when you take the mean of binary variables, each True evaluates to 1 and each False to 0, so the mean gives us the proportion of entries in `temp_all_months` that are lower than the 0.3 quantile:

In [ ]:
entries_less_than_q30 = temp_all_months["values"] < list_of_percentiles[0]
proportion_under_q30 = entries_less_than_q30.mean()
print(f"The proportion under {list_of_percentiles[0]} is {proportion_under_q30*100:.2f}%")

When we printed out the answer, we used some *number formatting*. This is written as `:.2f` within the curly brackets part of an f-string—this tells Python to display the number with two decimal places. You should also note that, as well as the mean given by `.mean()`, there are various other built-in functions like `.std()` for the standard deviation and `.var()` for the variance.

Now we can assess that between 1951 and 1980, 30% of observations for the temperature anomaly were smaller than –0.10, but between 1981 and 2010 only about two per cent of months are considered cold. That is a large change.

Let’s check whether we get a similar result for the number of observations that are larger than 0.11.

In [ ]:
proportion_over_q70 = (temp_all_months["values"] > list_of_percentiles[1]).mean()
print(f"The proportion over {list_of_percentiles[1]} is {proportion_over_q70*100:.2f}%")

## Python Walkthrough 1.7

**Calculating and understanding mean and variance**

The process of computing the mean and variance separately for each period and season separately would be quite tedious. We would prefer a way to cover all of them at once. Let's re-stack the data in a form where `season` is one of the columns and could take the values `DJF`, `MAM`, `JJA`, or `SON`. Let's also have a peek at the structure of the data while we're at it:

In [ ]:
temp_all_months = df.loc[:, "DJF":"SON"].stack().reset_index().rename(columns={"level_1": "season", 0: "values"})
temp_all_months["Period"] = pd.cut(
    temp_all_months["Year"],
    bins=[1921, 1950, 1980, 2010],
    labels=["1921—1950", "1951—1980", "1981—2010"],
    ordered=True,
)
temp_all_months.iloc[-135:-125]

Now we'll take the mean and variance at once. We're going to *group* our data using a `groupby` operation that we pass a list of the variables we'd like to group together; here that will be `"Period"` and `"season"`. The variable we'd like to apply the grouping to is `"values"` so we then filter down to just the `"values"` column. Finally we're going to do an *aggregation* using the `.agg` function and we'll pass that a list of functions we'd like to apply. We'll apply `np.mean` and `np.var`, which, respectively, take the mean and variance of any values they are applied to.

In [ ]:
grp_mean_var = temp_all_months.groupby(["season", "Period"])["values"].agg([np.mean, np.var])
grp_mean_var

We recognise that the variances seem to remain fairly constant across the first two periods, but they do increase markedly for the 1981—2010 period.

We can plot a line chart to see these changes graphically. (This type of chart is formally known as a ‘time-series plot’). One trick we can use here is that calling `.plot` on wide-format data (with many columns) is interpreted by **pandas** as you wanting to plot all of the columns you've selected. So, we can do a quick transform to our data and have **pandas** plot it in the way we'd like, with all of the different seasons represented.

To do this, we'll set `"Year"` and `"season"` as the index but then unstack them so that season forms the columns of our data. We'll then only select the `"values"` entries as we're not using `"Period"` right now:

In [ ]:
wide_format_data = temp_all_months.set_index(["Year", "season"]).unstack()["values"]
wide_format_data.head()

Now we can plot the data by calling `.plot` and passing that function the axis, here called `ax`, we would like it to use. We'll also add a horizontal line at zero and some labels.

In [ ]:
fig, ax = plt.subplots()
ax.axhline(0, color="black", alpha=0.3)
ax.annotate("1951—1980 average", xy=(0.68, -0.2), xycoords=("figure fraction", "data"))
wide_format_data.plot(linewidth=1, ax=ax)
ax.set_title(
    f"Average annual temperature anomaly in \n in the northern hemisphere (1880—{wide_format_data.index.max()})"
)
ax.set_ylabel("Annual temperature anomalies");